In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install ffmpeg-python pillow
!git clone https://github.com/soCzech/TransNetV2.git
%cd TransNetV2/inference

# LƯU Ý TRƯỚC KHI CHẠY
"""
Tải về máy và kéo thả để thêm các file trong folder "workspace/notebook/variable cho TransNetV2" vào thư mục TransNetV2 vừa clone (do Repo này bị lỗi)
- Thêm saved_model.pb vào TransNetV2/inference/transnetv2-weights
- Thêm 2 files còn lại (variables.index, variables.data-...) vào TransNetV2/inference/transnetv2-weights/variables
"""

Cloning into 'TransNetV2'...
remote: Enumerating objects: 362, done.
remote: Counting objects: 100% (88/88), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 362 (delta 71), reused 71 (delta 71), pack-reused 274 (from 1)
Receiving objects: 100% (362/362), 95.25 KiB | 9.52 MiB/s, done.
Resolving deltas: 100% (210/210), done.
Error downloading object: inference/transnetv2-weights/saved_model.pb (8ac2a52): Smudge error: Error downloading inference/transnetv2-weights/saved_model.pb (8ac2a52c5719690d512805b6eaf5ce12097c1d8860b3d9de245dcbbc3100f554): batch response: This repository exceeded its LFS budget. The account responsible for the budget should increase it to restore access.

Errors logged to /content/TransNetV2/.git/lfs/logs/20250816T023852.632962913.log
Use `git lfs logs last` to view the log.
error: external filter 'git-lfs filter-process' failed
fatal: inference/transnetv2-weights/saved_model.pb: smudge filter lfs failed
You can inspect what was checked out wit

'\nTải về máy và kéo thả để thêm các file trong folder "workspace/notebook/variable cho TransNetV2" vào thư mục TransNetV2 vừa clone (do Repo này bị lỗi)\n- Thêm saved_model.pb vào TransNetV2/inference/transnetv2-weights\n- Thêm 2 files còn lại (variables.index, variables.data-...) vào TransNetV2/inference/transnetv2-weights/variables\n'

In [3]:
import os
import cv2
import json
import glob
import ffmpeg
import torch
import numpy as np
from tqdm import tqdm
from transnetv2 import TransNetV2
import shutil
import json

In [4]:
dataset_dir = "/content/drive/Shareddrives/data_btc/video_btc"

all_video_paths = dict()

for folder in sorted(os.listdir(dataset_dir)):
    all_video_paths[folder] = dict()
print(all_video_paths)

for data_part in sorted(all_video_paths.keys()):
    data_part_path = f"{dataset_dir}/{data_part}/video"
    video_paths = sorted(glob.glob(f"{data_part_path}/*.mp4"))
    # print(video_paths)
    for video_path in video_paths:
        video_name = video_path.split("/")[-1].replace(".mp4", "")
        video_path_full = f"{data_part_path}/{video_path}"
        all_video_paths[data_part][video_name] = video_path

print(json.dumps(all_video_paths))

{'Videos_L21_a': {}, 'Videos_L22_a': {}, 'Videos_L23_a': {}, 'Videos_L24_a': {}, 'Videos_L25_a1': {}, 'Videos_L25_b': {}, 'Videos_L26_a': {}, 'Videos_L26_b': {}, 'Videos_L26_c': {}, 'Videos_L26_d': {}, 'Videos_L26_e': {}, 'Videos_L27_a': {}, 'Videos_L28_a': {}, 'Videos_L29_a': {}, 'Videos_L30_a': {}}
{"Videos_L21_a": {"L21_V001": "/content/drive/Shareddrives/data_btc/video_btc/Videos_L21_a/video/L21_V001.mp4", "L21_V002": "/content/drive/Shareddrives/data_btc/video_btc/Videos_L21_a/video/L21_V002.mp4", "L21_V003": "/content/drive/Shareddrives/data_btc/video_btc/Videos_L21_a/video/L21_V003.mp4", "L21_V005": "/content/drive/Shareddrives/data_btc/video_btc/Videos_L21_a/video/L21_V005.mp4", "L21_V006": "/content/drive/Shareddrives/data_btc/video_btc/Videos_L21_a/video/L21_V006.mp4", "L21_V007": "/content/drive/Shareddrives/data_btc/video_btc/Videos_L21_a/video/L21_V007.mp4", "L21_V008": "/content/drive/Shareddrives/data_btc/video_btc/Videos_L21_a/video/L21_V008.mp4", "L21_V009": "/content/

In [6]:
from transnetv2 import TransNetV2
model = TransNetV2()

json_save_dir = "/content/drive/Shareddrives/data/json_TransNetV2"
if not os.path.exists(json_save_dir):
    os.makedirs(json_save_dir)

TARGET_FOLDERS = [
    "Videos_L21_a",
    "Videos_L22_a",
    "Videos_L23_a",
    "Videos_L24_a",
    "Videos_L25_a1",
    "Videos_L25_b",
    "Videos_L26_a",
    "Videos_L26_b",
    "Videos_L26_c",
    "Videos_L26_d",
    "Videos_L26_e",
    "Videos_L27_a",
    "Videos_L28_a",
    "Videos_L29_a",
    "Videos_L30_a",
]

# ******* CHỈNH Ở ĐÂY *******
SELECTED_FOLDERS = ["Videos_L30_a"]

total_videos = sum(len(all_video_paths.get(f, {})) for f in SELECTED_FOLDERS)
pbar = tqdm(total=total_videos, desc="Detecting shots")

for folder in SELECTED_FOLDERS:
  video_paths = all_video_paths.get(folder, {})
  if not video_paths:
    print(f"[WARN] {folder} is empty or does not exist")
    continue
  output_dir = os.path.join(json_save_dir, folder)
  if not os.path.exists(output_dir):
    os.makedirs(output_dir)

  for video_name, video_path in video_paths.items():
    try:
      _, single_frame_pred, _ = model.predict_video(video_path)
      scenes = model.predictions_to_scenes(predictions=single_frame_pred)
      output_path = os.path.join(output_dir, f"{video_name}.json")
      with open(output_path, "w") as f:
        json.dump(scenes.tolist(), f)
    except Exception as e:
      print(f"[ERROR] {video_path}: {e}")
    finally:
      pbar.update(1)

pbar.close()
print("Done")

[TransNetV2] Using weights from /content/TransNetV2/inference/transnetv2-weights/.


Detecting shots:   0%|          | 0/96 [00:00<?, ?it/s]

[TransNetV2] Extracting frames from /content/drive/Shareddrives/data_btc/video_btc/Videos_L30_a/video/L30_V001.mp4
[TransNetV2] Processing video frames 2950/3162

Detecting shots:   1%|          | 1/96 [00:16<26:29, 16.73s/it]

[TransNetV2] Processing video frames 3162/3162
[TransNetV2] Extracting frames from /content/drive/Shareddrives/data_btc/video_btc/Videos_L30_a/video/L30_V002.mp4
[TransNetV2] Processing video frames 3950/4117

...

[TransNetV2] Processing video frames 5477/5477
[TransNetV2] Extracting frames from /content/drive/Shareddrives/data_btc/video_btc/Videos_L30_a/video/L30_V095.mp4
[TransNetV2] Processing video frames 7700/7739

Detecting shots:  99%|█████████▉| 95/96 [42:08<00:30, 30.37s/it]

[TransNetV2] Processing video frames 7739/7739
[TransNetV2] Extracting frames from /content/drive/Shareddrives/data_btc/video_btc/Videos_L30_a/video/L30_V096.mp4
[TransNetV2] Processing video frames 2200/2336

Detecting shots: 100%|██████████| 96/96 [42:21<00:00, 26.47s/it]

[TransNetV2] Processing video frames 2336/2336
Done


## Load Scene & Save Scene
Load the scene boundary data from the generated JSON files.

In [ ]:
# MODIFY HERE
json_save_dir = "/content/drive/Shareddrives/data/json_TransNetV2"
json_files = glob.glob(os.path.join(json_save_dir, "*.json"))

scene_data = {}

for json_file in json_files:
    video_id = os.path.splitext(os.path.basename(json_file))[0]
    with open(json_file, "r") as f:
        scene_data[video_id] = json.load(f)

print(f"Loaded scene data for {len(scene_data)} videos.")

Loaded scene data for 0 videos.


In [ ]:
# MODIFY HERE
keyframes_output_dir = "/content/drive/Shareddrives/data/keyframes_TransNetV2"
if not os.path.exists(keyframes_output_dir):
    os.makedirs(keyframes_output_dir)

for video_id, scenes in tqdm(scene_data.items(), desc="Processing videos"):
    video_path = all_video_paths.get(f"{video_id}.mp4")
    if video_path is None:
        print(f"Video path not found for video ID: {video_id}")
        continue

    video_output_dir = os.path.join(keyframes_output_dir, video_id)
    if not os.path.exists(video_output_dir):
        os.makedirs(video_output_dir)

    for scene_index, scene_boundary in enumerate(scenes):
        # This loop is just to prepare for the next steps.
        # Actual keyframe extraction will happen in the next subtask.
        pass


### Save frames

In [ ]:
import os
import cv2
from tqdm import tqdm

# MODIFY HERE
keyframes_output_dir = "/content/drive/Shareddrives/data/keyframes_TransNetV2"
if not os.path.listdir(keyframes_output_dir):
    os.makedirs(keyframes_output_dir)

sorted_video_ids = sorted(scene_data.keys())
# video_ids_to_process = sorted_video_ids[:10]  # Process first 10 after sorting

for video_id in tqdm(sorted_video_ids, desc="Processing videos"):
    scenes = scene_data.get(video_id)
    if scenes is None:
        print(f"Scene data not found for video ID: {video_id}")
        continue

    video_path = all_video_paths.get(f"{video_id}.mp4")
    if video_path is None:
        print(f"Video path not found for video ID: {video_id}")
        continue

    video_output_dir = os.path.join(keyframes_output_dir, video_id)
    os.makedirs(video_output_dir, exist_ok=True)

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Could not open video file {video_path}")
        continue

    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    if total_frames <= 0:
        print(f"Warning: {video_path} has invalid frame count ({total_frames}). Skipping.")
        cap.release()
        continue

    print(f"Processing video: {video_id}")

    # Collect unique indices (start, middle, end) across all scenes
    indices_to_extract = set()
    for scene_index, (start_frame, end_frame) in enumerate(scenes):
        # Ensure order (handle bad input where end < start)
        s = int(min(start_frame, end_frame))
        e = int(max(start_frame, end_frame))
        # Clamp to valid range
        s = max(0, min(s, total_frames - 1))
        e = max(0, min(e, total_frames - 1))
        m = s + (e - s) // 2  # middle

        # Dedup by using a set
        indices_to_extract.update((s, m, e))

        print(f"  Scene {scene_index}: Boundary [{start_frame}, {end_frame}] "
              f"-> clamped [{s}, {e}], frames: {sorted({s, m, e})}")

    # Read & save
    for frame_index in sorted(indices_to_extract):
        output_filename = f"{frame_index:06d}.jpg"
        output_path = os.path.join(video_output_dir, output_filename)

        if os.path.exists(output_path):
            # already saved (handy for re-runs)
            continue

        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_index)
        ret, frame = cap.read()

        if ret and frame is not None:
            cv2.imwrite(output_path, frame)
            # print(f"    Saved frame: {output_path}")  # uncomment if you want logs
        else:
            print(f"    Warning: Could not read frame {frame_index} from {video_path}")

    cap.release()


# Convert JSON Shot Boundary output from TransNetV2 to PySceneDetect Format

In [ ]:
import json
import os
import csv
from pathlib import Path

FPS = 25

def seconds_to_timecode(seconds):
    """Convert seconds to timecode HH:MM:SS.MS"""
    hours = int(seconds // 3600)
    minutes = int((seconds % 3600) // 60)
    secs = int(seconds % 60)
    milliseconds = int((seconds % 1) * 1000)
    return f"{hours:02d}:{minutes:02d}:{secs:02d}.{milliseconds:03d}"

def convert_json_to_csv(json_path, output_dir):
    # Ensure output_dir exists
    os.makedirs(output_dir, exist_ok=True)

    # Determine output CSV path with the same name as JSON
    json_path = Path(json_path)
    output_filename = json_path.stem + ".csv"
    csv_path = os.path.join(output_dir, output_filename)

    # Load JSON
    with open(json_path, "r") as f:
        shots = json.load(f)

    # Write CSV
    with open(csv_path, "w", newline='') as f:
        writer = csv.writer(f)
        writer.writerow([
            "Scene Number",
            "Start Frame",
            "Start Timecode",
            "Start Time (seconds)",
            "End Frame",
            "End Timecode",
            "End Time (seconds)",
            "Length (frames)",
            "Length (timecode)",
            "Length (seconds)"
        ])

        for idx, (start_frame, end_frame) in enumerate(shots):
            scene_number = idx + 1

            start_sec = start_frame / FPS
            end_sec = end_frame / FPS
            length_frames = end_frame - start_frame + 1
            length_sec = length_frames / FPS

            start_tc = seconds_to_timecode(start_sec)
            end_tc = seconds_to_timecode(end_sec)
            length_tc = seconds_to_timecode(length_sec)

            writer.writerow([
                scene_number,
                start_frame + 1,  # Convert to 1-based indexing
                start_tc,
                round(start_sec, 2),
                end_frame + 1,
                end_tc,
                round(end_sec, 2),
                length_frames,
                length_tc,
                round(length_sec, 2)
            ])

    print(f"✅ Saved: {csv_path}")

if __name__ == "__main__":
    shots_dir = "/home/nhantt/AIC2024/dict/TransNetV2_keyframe_extraction"
    output_dir = "/home/nhantt/AIC2024/data4t_link/dict/TransNetV2_keyframe_extraction_PySceneDetect_format"

    json_files = sorted([
        os.path.join(shots_dir, f)
        for f in os.listdir(shots_dir)
        if f.endswith(".json")
    ])

    # Modify to run specific files
    json_files = json_files[11:]

    for json_file in json_files:
        print(f"Processing: {json_file}")
        convert_json_to_csv(json_file, output_dir)

    # To process all files, uncomment:
    # for json_file in tqdm(json_files):
    #     convert_json_to_csv(json_file, output_dir)
